In [33]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()

In [24]:
print(melb_df['Address'].nunique()) # подчет количества уникальных значений
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

13378
2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


In [26]:
# на вход данной функции поступает адрес:
def get_street_type(adress):
    # создаем список географических пометок
    exclude_list = ['N','S', 'W', 'E']
    # метод split() разбивает строку на слова по пробелу. В рузльтате
    # получаем список слов в строке и заносим его в переменную adress_list
    adress_list = adress.split(' ')
    # Обрезаем список, оставляя в нем только последний элемент, потенциальный
    # подтип улицы, и заносим в переменную street_type
    street_type = adress_list[-1]
    # делаем провекру, что полученный подтип явялется географической пометкой,
    # Для этого проверяем его на наличие в списке exclude_list
    if street_type in exclude_list:
        # если переменная street_type явяляется географической пометкой,
        # переопределяем ее на второй элемент с конца списка adress_list.
        street_type = adress_list[-2]
        # возвращаем переменную street_typpe, в которой хранится подтип улицы
    return street_type
    

Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию get_street_type в аргумент метода столбца apply(). В результате получим объект Series, который положим в переменную street_types:

In [27]:
street_type = melb_df['Address'].apply(get_street_type)
display(street_type)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

In [28]:
# проверим количество уникальных значений подтипа улиц
print(street_type.nunique())
# посмотрим частоту уникальных значений
display(street_type.value_counts())
# топ 10 самых частых оставим, а остальные нозовем other 


56


Address
St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Co

In [29]:
popular_stypes = street_type.value_counts().nlargest(10).index 
# к результату метода value_counts() применим метод nlargest(10) - т.е.
# возвращаем 10 наибольших значений из value_counts().
# метод index - вытаскиваем индексы, а не их значения value_counts()
print(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object', name='Address')


Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'

In [30]:
melb_df['StreetType'] = street_type.apply(lambda x: x if x in popular_stypes
                                          else 'other')
display(melb_df['StreetType'])
print(melb_df['StreetType'].nunique()) # проверим количество уникальных значений
# теперь на не нужно хранить столбец Address для анализа, т.к. есть столбцы
# с обозначением долготы и широты и есть типы улиц
melb_df = melb_df.drop('Address', axis=1)

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

11


Практика!!!
1. Ранее, в задании 3.3, мы создали признак WeekdaySale в таблице melb_df — день недели продажи. Из полученных в задании результатов можно сделать вывод, что объекты недвижимости в Мельбурне продаются преимущественно по выходным (суббота и воскресенье).
Напишите функцию get_weekend(weekday), которая принимает на вход элемент столбца WeekdaySale и возвращает 1, если день является выходным, и 0 — в противном случае, и создайте столбец Weekend в таблице melb_df с помощью неё.

Примените эту функцию к столбцу и вычислите среднюю цену объекта недвижимости, проданного в выходные дни. Результат округлите до целого числа.

In [35]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
# dayfirst - указание, что день идет первым
display(melb_df['Date'])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

In [36]:
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[
    (melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)
    ].shape[0]
print(weekend_count)

12822


In [37]:
def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else:
        return 0

In [39]:
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
display(melb_df['Weekend'])

0        1
1        0
2        1
3        1
4        1
        ..
13575    1
13576    1
13577    1
13578    1
13579    1
Name: Weekend, Length: 13580, dtype: int64

In [40]:
print(round(melb_df[melb_df['Weekend'] == 1]['Price'].mean()))

1081199


2. Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.
Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. Ответ округлите до одного знака после точки-разделителя.

In [46]:
popular_Seller = melb_df['SellerG'].value_counts().nlargest(49).index 
print(popular_Seller)
melb_df['PopSeller'] = melb_df['SellerG'].apply(lambda x: x if x in popular_Seller
                                          else 'other')

Index(['Nelson', 'Jellis', 'hockingstuart', 'Barry', 'Ray', 'Marshall',
       'Buxton', 'Biggin', 'Brad', 'Fletchers', 'Woodards', 'Jas', 'Greg',
       'McGrath', 'Sweeney', 'Noel', 'Miles', 'RT', 'Gary', 'Harcourts',
       'Hodges', 'YPA', 'Stockdale', 'Village', 'Kay', 'Raine', 'Williams',
       'Love', 'Douglas', 'Chisholm', 'RW', 'Rendina', 'HAR', 'O'Brien', 'C21',
       'Collins', 'Cayzer', 'Eview', 'Purplebricks', 'Philip', 'Buckingham',
       'Bells', 'Thomson', 'Nick', 'Alexkarbon', 'McDonald', 'Burnham',
       'Moonee', 'LITTLE'],
      dtype='object', name='SellerG')


In [47]:
display(melb_df['PopSeller'].value_counts())

PopSeller
Nelson           1565
Jellis           1316
other            1199
hockingstuart    1167
Barry            1011
Ray               701
Marshall          659
Buxton            632
Biggin            393
Brad              342
Woodards          301
Fletchers         301
Jas               243
Greg              239
McGrath           222
Sweeney           216
Noel              205
Miles             196
RT                184
Gary              170
Harcourts         168
Hodges            157
YPA               154
Stockdale         150
Village           125
Kay               119
Raine             116
Williams          111
Love              109
Douglas            97
Chisholm           77
RW                 70
Rendina            66
HAR                62
O'Brien            61
C21                57
Collins            56
Cayzer             52
Eview              51
Purplebricks       51
Philip             48
Buckingham         46
Bells              44
Thomson            42
Nick               40


In [49]:
print(
    round(
        melb_df[melb_df['PopSeller'] == 'Nelson']['Price'].min() 
        / melb_df[melb_df['PopSeller'] == 'other']['Price'].min(), 1))

1.3


3. Представьте, что вы занимаетесь подготовкой данных о вакансиях с платформы hh.ru. В вашем распоряжении имеется таблица, в которой с помощью парсинга собраны резюме кандидатов. В этой таблице есть текстовый столбец «Опыт работы». Пример такого столбца представлен ниже в виде объекта Series. Структура текста в столбце фиксирована и не может измениться.

Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.

In [50]:
def get_experience(arg):
    month_key_words = ['месяц', 'месяцев', 'месяца']
    year_key_words = ['год', 'лет', 'года']
    args_splited = arg.split(' ')
    month = 0
    year = 0
    for i in range(len(args_splited)):
        if args_splited[i] in month_key_words:
            month = args_splited[i-1]
        if args_splited[i] in year_key_words:
            year = args_splited[i-1]
    return int(year)*12 + int(month)